# Utilisation du modèle TensorRT optimisé

Dans ce Notebook, nous allons étudier comment utiliser un modèle optimisé avec TensorRT à l'aide de Cuda.

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import shutil

from tensorflow import keras
from matplotlib import pyplot as plt

### Chargement du moteur (engine)

On commence par créer une instance de logger qui va nous permettre d'afficher les informations pendant l'utilisation du moteur.

In [ ]:
import tensorrt as trt

# Construction de la class du logger
class MyLogger(trt.ILogger):
    def __init__(self):
        trt.ILogger.__init__(self)

    def log(self, severity, msg):
        print("%s : %s" %(severity,msg))
        pass

 Ensuite, on créé un ``runtime`` (environnement de travail) pour notre modèle puis on charge le moteur à l'aide de la fonction ``deserialize_cuda_engine`` que nous porpose ce runtime.

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit

PRECISION = trt.float32

logger = MyLogger()
runtime = trt.Runtime(logger)
trt.init_libnvinfer_plugins(logger, namespace="")
    
with open("tfmodel_ssd_mobilenet_v2_320x320_coco17_tpu-8/model.engine", "rb") as f:
    engine = runtime.deserialize_cuda_engine(f.read())

 ### Création du contexte

Ensuite, il faut créer un contexte nécessaire pour lancer le modèle. Ce contexte va allouer une certaine quantité de mémoire afin de pouvoir travailler avec notre mmoteur.

In [ ]:
context = engine.create_execution_context()

### Allocation de l'espace mémoire hôte et GPU

Nous allons devoir allouer de l'espace en mémoire sur **l'hote** et sur le **GPU** pour réaliser des transferts de données entre eux.
Commençons par regarder combien de mémoire nous avons en GPU :

In [ ]:
cuda.mem_get_info()

Pour transférer des données entre l'hôte et le GPU, celles-ci doivent être transférées depuis un espace mémoire paginé vérrouillé de l'hôte. On va donc devoir réserver un espace mémoire sur l'hôte et le vérrouiller (cet espace ne pourra pas servir à autre chose et donc ne pourra pas être utlisé par un autre processus).

Les étapes à suivre pour utliser notre moteur sont les suivantes :
- Réserver un espace mémoire vérrouillé sur l'hote pour y placer l'entrée (**input_host_mem**) et la sortie du modèle (**output_host_mem**)
- Réserver un espace mémoire dans le GPU pour y placer ces mêmes informations (**input_device_mem** et **output_device_mem**)
- Placer les données d'entrées du modèle (notre image) dans l'espace mémoire hôte vérrouillé qu'on a pris soin de réserver au préalable (**input_host_mem**)
- Transférer le contenu de cet espace mémoire vers la mémoire du GPU (**input_device_mem**)
- Lancer les calculs avec le modèle TensorRT qu'on a défini ; les résultats seront écrits dans la mémoire du GPU (**output_device_mem**)
- Transférer les résultats de la mémoire GPU (**output_device_mem**) vers la mémoire vérrouillées de l'hote (**output_host_mem**)

On peut trouver des informations sur le format des tenseurs en entrée et en sortie de notre moteur :

In [ ]:
for binding in engine:
    print(engine.get_binding_shape(binding))
    print(trt.nptype(engine.get_binding_dtype(binding)))

On va donc utiliser ces informations pour allouer les espaces en mémoire hôte et GPU :

In [ ]:
# Réservation de la mémoire pour l'entrée
size_input = trt.volume(engine.get_binding_shape(0))* engine.max_batch_size
input_host_mem = cuda.pagelocked_empty(size_input, trt.nptype(PRECISION))
input_device_mem = cuda.mem_alloc(input_host_mem.nbytes)

In [ ]:
# Réservation de la mémoire pour les sorties
output_device_mem = [];
format_sorties = [];
types_sorties = [];

for i in range(engine.num_bindings):
    if not engine.binding_is_input(i):
        size_output = trt.volume(engine.get_binding_shape(i))* engine.max_batch_size
        output_host_mem = cuda.pagelocked_empty(size_output, trt.nptype(PRECISION))
        output_device_mem.append(cuda.mem_alloc(output_host_mem.nbytes))
        format_sorties.append(engine.get_binding_shape(i))
        types_sorties.append(trt.nptype(engine.get_binding_dtype(i)))

Il faut maintenant récupérer les adresses mémoires GPU. Ces adresses seront utiles au contexte de TensorRT précédemment ouvert pour pouvoir exécuter le modèle.

In [ ]:
# Récupère les adresses en GPU des buffers entrées / sorties
binding_entree = int(input_device_mem)

binding_sorties = []
for output_ in output_device_mem:
    binding_sorties.append(int(output_))

### Exécution d'une prédiction

On commence par récupérer une image du dataset :

In [ ]:
image = tf.keras.preprocessing.image.load_img("models/research/object_detection/test_images/image2.jpg",
                                             target_size=(320, 320))
image

Puis on convertit cette image en tenseur :

In [ ]:
image = tf.keras.preprocessing.image.img_to_array(image)
image.shape

On copie maintenant ce tenseur dans l'espace mémoire d'entrée de l'hôte. Le tenseur est applati à l'aide de la fonction ``ravel()``:

In [ ]:
image = np.expand_dims(image,axis=0)                          # (1,320,320,3)
np.copyto(input_host_mem,image.ravel())

Transfert les données de l'image vers la mémoire GPU (transfert Host => Device) :

In [ ]:
cuda.memcpy_htod(input_device_mem, input_host_mem)

Exécution du modèle :

In [ ]:
bindings = [binding_entree, binding_sorties[0],binding_sorties[1],binding_sorties[2],binding_sorties[3]]

In [ ]:
bindings

In [ ]:
context.execute(batch_size=1,bindings=bindings)

On transfert le résultat stocké en mémoire GPU vers la mémoire destination de l'hôte :

In [ ]:
output_device_mem

In [ ]:
output_host_mem = []
for i in range(len(output_device_mem)):
    output_host_mem.append(np.zeros(format_sorties[i],types_sorties[i]))

for i in range(len(output_host_mem)):
    cuda.memcpy_dtoh(output_host_mem[i], output_device_mem[i])
output_host_mem

## Visualisation du résultat

In [ ]:
from PIL import ImageDraw
from PIL import Image
from io import BytesIO


def draw_bbox_and_label_in_image(img, boxes, num_detections, box_width=3):
    """
    Draw bounding boxes and class labels in images
    :param img: PIL Image or np arrays
    :param boxes: in size [num_detections, 4], contains xys or boxes
    :param box_width: the width of boxes
    :return: Image
    """

    draw = ImageDraw.Draw(img)
    width, height = img.size

    for i in range(num_detections):
        ymin, xmin, ymax, xmax = boxes[i]

        ymin = int(ymin * height)
        ymax = int(ymax * height)
        xmin = int(xmin * width)
        xmax = int(xmax * width)

        class_color = "LimeGreen"

        draw.line([(xmin, ymin), (xmax, ymin), (xmax, ymax), (xmin, ymax), (xmin, ymin)], width=box_width, fill=class_color)

    return img

img_data = tf.io.gfile.GFile('models/research/object_detection/test_images/image2.jpg', 'rb').read()
image = Image.open(BytesIO(img_data))

draw_image = draw_bbox_and_label_in_image(image, output_host_mem[1][0,0:3,:],3)
draw_image

Enfin pour terminer, on libère la mémoire allouée :

In [ ]:
input_device_mem.free()
for i in range(len(output_device_mem)):
    output_device_mem[i].free()
del input_host_mem
del output_host_mem
del context
del engine